# MTH3302: Méthodes probabilistes et statistiques pour l'I.A.
## Projet

- Vincent Haney
- Félix Pelletier
- Charles-François St-Cyr (275354)
- Mikael Vaillant (2029994)


### Librairies

In [753]:
import Pkg;
packages = ["CSV", "DataFrames", "Distributions", "Gadfly", "MLBase", "Random", "Statistics", "Colors", "GLM", "LinearAlgebra", "Combinatorics", "CategoricalArrays","Distances","NearestNeighbors"];
# peut aps être mis dans une fonction. using doit être top-level
for package in packages
    try
        @eval using $(Symbol(package))
    catch e
        Pkg.add(package)
        @eval using $(Symbol(package))
    end
end
Pkg.update()

include("./library/main.jl")
include("./library/normalize.jl")

    Updating registry at `C:\Users\vaill\.julia\registries\General.toml`
  No Changes to `C:\Users\vaill\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\vaill\.julia\environments\v1.8\Manifest.toml`


normalize (generic function with 9 methods)

### Chargement des données (partiel pour l'exploration)

In [754]:
data = CSV.read("./data/train.csv", DataFrame)
test = CSV.read("./data/test.csv", DataFrame)

train, valid = partitionData(data, 0.75);

first(train, 5)

Row,ID,cut,color,clarity,depth,table,price,x,y,z
,Int64,String15?,String1,String7,Float64?,Float64,Int64,Float64,Float64?,Float64
1,1,missing,E,SI2,60.2,60.0,5340,6.46,6.5,3.9
2,4,Ideal,G,VS2,61.7,57.0,720,4.41,4.38,2.71
3,6,Premium,D,VVS2,61.5,60.0,925,4.36,4.42,2.7
4,7,Very Good,D,SI1,60.2,61.0,571,4.39,4.42,2.65
5,9,Fair,I,VS1,64.4,59.0,1229,5.3,5.2,3.38


### Visualisation des données

In [755]:
# plot(data, x = :price, Geom.histogram())

In [756]:
println("Format des données (entrainement et validation): ", names(data))

missing_counts = collect(count(ismissing,col) for col in eachcol(data))
println("Nombre de valeurs manquantes dans les différentes colonnes: ", missing_counts)

# zero_counts = [count(x -> x == 0, skipmissing(data[:, col])) for col in names(data)]
zero_counts = hcat([0,0,0,0]',hcat([count(x -> x < 0.05, skipmissing(data[:, col])) for col in [:depth,:table,:price,:x, :y, :z]]...))

println("Nombre de valeurs nulles dans les différentes colonnes: ",zero_counts)

println("Taille de l'ensemble de donnée (entrainement et validation): ",size(data,1))

println("Format des données test: ", names(test))

missing_counts = collect(count(ismissing,col) for col in eachcol(test))
println("Nombre de valeurs manquantes dans les différentes colonnes: ", missing_counts)

zero_counts = hcat([0,0,0,0]',hcat([count(x -> x < 0.05, skipmissing(test[:, col])) for col in [:depth,:table,:x, :y, :z]]...))
println("Nombre de valeurs nulles dans l'ensemble test: ",zero_counts)

println("Taille de l'ensemble de donnée (test): ",size(test,1))



Format des données (entrainement et validation): ["ID", "cut", "color", "clarity", "depth", "table", "price", "x", "y", "z"]
Nombre de valeurs manquantes dans les différentes colonnes: [0, 3806, 0, 0, 2201, 0, 0, 0, 2200, 0]
Nombre de valeurs nulles dans les différentes colonnes: [0 0 0 0 13 0 0 4 3 15]
Taille de l'ensemble de donnée (entrainement et validation): 40455
Format des données test: ["ID", "cut", "color", "clarity", "depth", "table", "x", "y", "z"]
Nombre de valeurs manquantes dans les différentes colonnes: [0, 1194, 0, 0, 800, 0, 0, 800, 0]
Nombre de valeurs nulles dans l'ensemble test: [0 0 0 0 5 0 4 4 5]
Taille de l'ensemble de donnée (test): 13485


In [757]:
function fillDataFromAverage(data::DataFrame; is_test::Bool = false)
    avgCut = mode(dropmissing(data, :cut).cut)
    avgColor = mode(dropmissing(data, :color).color)
    avgClarity = mode(dropmissing(data, :clarity).clarity)
    avgDepth = mean(dropmissing(data, :depth).depth)
    avgTable = mean(dropmissing(data, :table).table)
    avgX = mean(dropmissing(data, :x).x)
    avgY = mean(dropmissing(data, :y).y)
    avgZ = mean(dropmissing(data, :z).z)
    # avgVol = mean(dropmissing(data, :volume).volume)

    replace!(data.cut, missing => avgCut)
    replace!(data.color, missing => avgColor)
    replace!(data.clarity, missing => avgClarity)
    replace!(data.depth, missing => avgDepth)
    replace!(data.table, missing => avgTable)
    replace!(data.x, missing => avgX)
    replace!(data.y, missing => avgY)
    replace!(data.z, missing => avgZ)
    # replace!(data.volume, missing => avgVol)

    if (!is_test)
        avgPrice = mean(dropmissing(data, :price).price)
        replace!(data.price, missing => avgPrice)
    end
end

fillDataFromAverage (generic function with 1 method)

#### NOTES - voici quelques informations importantes découvertes

1) enlever les variables depth, table et cut aide à avoir un meilleur RMSE. Peut-être possible de les intégrer en réduisant leur poids? 
2) Le nombre de voisins peut être ajusté pour obtenir un meilleur RMSE


#### Prédictions pour les données pour lesquelles toutes les lignes pleines

Plusieurs manipulations sont effectuées sur les matrices pour permettre d'appliquer l'algorithme knn. 

1. Sélectionner toutes les lignes complètes du DataFrame
2. Modifier les valeurs qualitatives en valeurs quantitatives
3. Centrer et réduire les colonnes


In [758]:
# -------------------------------------------
# Manipulations sur le fichier d'entrainement
# -------------------------------------------

# Copie de train et enregistrement des prix actuel
train_ = deepcopy(train)
train_ = train_[completecases(train_), :]
price_train = select(train_, (:price))

# Conversion des colonnes qualitatives en colonnes quantitatives - cut a été enlevé ici
convertQualitativeToQuantitative(train_, :color)
convertQualitativeToQuantitative(train_, :clarity)

# Sélection de toutes les colonnes quantitatives
train_no_empty = select(train_, Not(["price","cut","ID","color","clarity","depth","table"]))

# Normalisation des colonnes de la matrice
for col in names(train_no_empty[:,1:end])
    train_no_empty[!, col] = (train_no_empty[!, col] .- mean(train_no_empty[!, col])) ./ std(train_no_empty[!, col])
end

# ------------------------------------------
# Manipulations sur le fichier de validation
# ------------------------------------------

# On sélectionne toutes les données qui possèdent des lignes complètes et qui ne sont pas nulles
valid_no_empty = deepcopy(valid)
valid_no_empty = valid_no_empty[completecases(valid_no_empty), :]
valid_no_empty = valid_no_empty[valid_no_empty.z .>= 0.05, :]

test_no_empty = deepcopy(test)
test_no_empty = test_no_empty[completecases(test_no_empty), :]
test_no_empty = test_no_empty[test_no_empty.z .>= 0.05, :]

# Enregistrement du prix actuel et du ID pour le calcul du RMSE 
price_true_no_empty = select(valid_no_empty, (:price))
prediction_no_empty = select(valid_no_empty, (:ID))

# price_test_no_empty = select(test_no_empty, (:price))
prediction_test_no_empty = select(test_no_empty, (:ID))

# Conversion des colonnes qualitatives en colonnes quantitatives - cut a été enlevé ici
convertQualitativeToQuantitative(valid_no_empty, :color)
convertQualitativeToQuantitative(valid_no_empty, :clarity)

convertQualitativeToQuantitative(test_no_empty, :color)
convertQualitativeToQuantitative(test_no_empty, :clarity)

# Sélection de toutes les colonnes quantitatives
valid_no_empty = select(valid_no_empty, Not(["price","cut","ID","color","clarity","depth","table"]))

test_no_empty = select(test_no_empty, Not(["cut","ID","color","clarity","depth","table"]))

# Normalisation des colonnes de la matrice
for col in names(valid_no_empty[:,1:end])
    valid_no_empty[!, col] = (valid_no_empty[!, col] .- mean(valid_no_empty[!, col])) ./ std(valid_no_empty[!, col])
end

for col in names(test_no_empty[:,1:end])
    test_no_empty[!, col] = (test_no_empty[!, col] .- mean(test_no_empty[!, col])) ./ std(test_no_empty[!, col])
end

# valid_no_empty
# valid_y_empty[!, 1:3] = valid_y_empty[!, 1:3].*3
# train_y_empty[!, 1:3] = train_y_empty[!, 1:3].*3


In [759]:
# Transformation des données pour le knn
Mat_train = Matrix(train_no_empty)'
Mat_valid = Matrix(valid_no_empty)'
Mat_test = Matrix(test_no_empty)'

# Génération de l'arbre de données
kdtree = KDTree(Mat_train)

# Nombre de plus proches voisins à utiliser
k = 5

# Algo knn pour valid
idxs, dists = knn(kdtree, Mat_valid, k, true)

prediction = zeros(size(idxs))
for i in 1:size(idxs,1)
    prediction[i] =  mean(Matrix(price_train)[idxs[i]])
end

prediction_no_empty.price = prediction;

# Algo knn pour test
idxs, dists = knn(kdtree, Mat_test, k, true)

prediction = zeros(size(idxs))
for i in 1:size(idxs,1)
    prediction[i] =  mean(Matrix(price_train)[idxs[i]])
end

prediction_test_no_empty.price = prediction;

# Enregistrement des prédictions de prix avec les IDs pour RMSE


#### Prédictions pour les données manquants des valeurs en y et depth

In [760]:
# -------------------------------------------
# Manipulations sur le fichier d'entrainement
# -------------------------------------------

# Sélection de toutes les colonnes quantitatives
train_y_empty = select(train_, Not(["price","cut","ID","color","clarity","depth","table","y"]))

# Normalisation des colonnes de la matrice
for col in names(train_y_empty[:,1:end])
    train_y_empty[!, col] = (train_y_empty[!, col] .- mean(train_y_empty[!, col])) ./ std(train_y_empty[!, col])
end

# ------------------------------------------
# Manipulations sur le fichier de validation
# ------------------------------------------

#En general, y et depth sont manquants en même temps, on va les évaluer en même temps avec les deux colonnes manquantes 
valid_y_empty = deepcopy(valid)
valid_y_empty = valid_y_empty[ismissing.(valid_y_empty[!, "depth"]) .| ismissing.(valid_y_empty[!, "y"]) .| ismissing.(valid_y_empty[!, "cut"]), :]

test_y_empty = deepcopy(test)
test_y_empty = test_y_empty[ismissing.(test_y_empty[!, "depth"]) .| ismissing.(test_y_empty[!, "y"]) .| ismissing.(test_y_empty[!, "cut"]), :]

# Enregistrement du prix actuel et du ID pour le calcul du RMSE 
price_true_y_empty = select(valid_y_empty, (:price))
prediction_y_empty = select(valid_y_empty, (:ID))

# price_test_y_empty = select(test_y_empty, (:price))
prediction_test_y_empty = select(test_y_empty, (:ID))

# Conversion des colonnes qualitatives en colonnes quantitatives - cut a été enlevé ici
convertQualitativeToQuantitative(valid_y_empty, :color)
convertQualitativeToQuantitative(valid_y_empty, :clarity)

convertQualitativeToQuantitative(test_y_empty, :color)
convertQualitativeToQuantitative(test_y_empty, :clarity)

# Sélection de toutes les colonnes quantitatives
valid_y_empty = select(valid_y_empty, Not(["price","cut","ID","color","clarity","depth","table","y"]))

test_y_empty = select(test_y_empty, Not(["cut","ID","color","clarity","depth","table","y"]))

# Normalisation des colonnes de la matrice
for col in names(valid_y_empty[:,1:end])
    valid_y_empty[!, col] = (valid_y_empty[!, col] .- mean(valid_y_empty[!, col])) ./ std(valid_y_empty[!, col])
end

for col in names(test_y_empty[:,1:end])
    test_y_empty[!, col] = (test_y_empty[!, col] .- mean(test_y_empty[!, col])) ./ std(test_y_empty[!, col])
end

# Changement de poids des différentes variables
# valid_y_empty[!, 1:2] = valid_y_empty[!, 1:2].*2
# train_y_empty[!, 1:2] = train_y_empty[!, 1:2].*2

In [761]:
# Transformation des données pour le knn
Mat_train = Matrix(train_y_empty)'
Mat_valid = Matrix(valid_y_empty)'
Mat_test = Matrix(test_y_empty)'
kdtree = KDTree(Mat_train)

# Nombre de plus proches voisins à utiliser
k = 5

# Algo knn validation
idxs, dists = knn(kdtree, Mat_valid, k, true)

prediction = zeros(size(idxs))
for i in 1:size(idxs,1)
    prediction[i] =  mean(Matrix(price_train)[idxs[i]])
end

# Enregistrement des prédictions de prix avec les IDs pour RMSE
prediction_y_empty.price = prediction;

# Algo knn test
idxs, dists = knn(kdtree, Mat_test, k, true)

prediction = zeros(size(idxs))
for i in 1:size(idxs,1)
    prediction[i] =  mean(Matrix(price_train)[idxs[i]])
end

# # Enregistrement des prédictions de prix avec les IDs pour RMSE
prediction_test_y_empty.price = prediction;



#### Prédictions pour les données nulles et abberrantes

En raison du très faible nombre de valeurs nulles ou aberrantes, le prix moyen des diamants a été utilisé

In [762]:
# ------------------------------------------
# Manipulations sur le fichier de validation
# ------------------------------------------

#Il n'y a jamais un zéro et des valeurs manquantes
valid_0 = deepcopy(valid)
valid_0 = valid_0[completecases(valid_0), :]
valid_0 = valid_0[valid_0.z .< 0.05, :]

test_0 = deepcopy(test)
test_0 = test_0[completecases(test_0), :]
test_0 = test_0[test_0.z .< 0.05, :]

price_true_0 = select(valid_0, (:price))
prediction_0 = select(valid_0, (:ID))

# price_test_0 = select(test_0, (:price))
prediction_test_0 = select(test_0, (:ID))

prediction = mean(train[!,"price"])

prediction_0.price = prediction*ones(size(prediction_0,1))
prediction_test_0.price = prediction*ones(size(test_0,1));


#### Calcul du RMSE pour tous les groupes de validation

In [763]:
prediction_valid = sort(vcat(prediction_0,prediction_y_empty,prediction_no_empty))[:,"price"]

# valid
RMSE = sqrt(mean((prediction_valid .- valid[:,"price"]).^2))

# prediction_y_empty

637.1972390014195

In [764]:
prediction_test = sort(vcat(prediction_test_0,prediction_test_y_empty,prediction_test_no_empty))[:,"price"]
# test.ID
CSV.write("./results/predictions_knn.csv", DataFrame(ID = test.ID, price = prediction_test));

#### Ancienne version

In [765]:
# train₁ = deepcopy(train)
# valid₁ = deepcopy(valid)
# valid2 = Matrix(deepcopy(valid))
# test₁ = deepcopy(test)

# fillDataFromAverage(train₁)
# fillDataFromAverage(valid₁)
# fillDataFromAverage(test₁, is_test = true)

# convertQualitativeToQuantitative(train₁, :color)
# # convertQualitativeToQuantitative(train₁, :cut)
# convertQualitativeToQuantitative(train₁, :clarity)

# convertQualitativeToQuantitative(valid₁, :color)
# # convertQualitativeToQuantitative(valid₁, :cut)
# convertQualitativeToQuantitative(valid₁, :clarity)

# convertQualitativeToQuantitative(test₁, :color)
# # convertQualitativeToQuantitative(test₁, :cut)
# convertQualitativeToQuantitative(test₁, :clarity)

# price_train = select(train₁, (:price))
# price_true = select(valid₁, (:price))
# test_ids = select(test₁, (:ID))

# train₁ = select(train₁, Not(["price","cut","ID","color","clarity","y","depth","table"]))
# valid₁ = select(valid₁, Not(["price","cut","ID","color","clarity","y","depth","table"]))
# test₁ = select(test₁, Not(["cut","ID","color","clarity","y","depth","table"]))

# for col in names(train₁[:,1:3])
#     train₁[!, col] = (train₁[!, col] .- mean(train₁[!, col])) ./ std(train₁[!, col])
# end

# for col in names(valid₁[:,1:3])
#     valid₁[!, col] = (valid₁[!, col] .- mean(valid₁[!, col])) ./ std(valid₁[!, col])
# end

# for col in names(test₁[:,1:3])
#     test₁[!, col] = (test₁[!, col] .- mean(test₁[!, col])) ./ std(test₁[!, col])
# end



# first(train₁,5)

In [766]:

# Mat_train = Matrix(train₁)'
# k = 4
# Mat_valid = Matrix(valid₁)'

# kdtree = KDTree(Mat_train)
# idxs, dists = knn(kdtree, Mat_valid, k, true)

# prediction = zeros(size(idxs))
# for i in 1:size(idxs,1)
#     prediction[i] =  mean(Matrix(price_train)[idxs[i]])
# end

# RMSE = sqrt(mean((prediction .- Matrix(price_true)).^2))


In [767]:
# Afin de vérifier que les plus grosses erreurs ne sont pas provoquées par les valeurs manquantes
# On regarde dans le 1/5 des erreurs les plus grande. Si ratio_missing >> 0.2, il y a un problème

# err_indivi = sqrt.(((prediction .- Matrix(price_true))[:,1]).^2)
# id_err_max = sortperm(err_indivi)[end:-1:1]
# data_err_max = valid2[id_err_max[1:div(size(valid2,1),5)],:]

# missing_counts1 = collect(count(ismissing,col) for col in eachcol(data_err_max))

# missing_counts2 = collect(count(ismissing,col) for col in eachcol(valid2))

# ratio_missing = round.([missing_counts1[2],missing_counts1[5],missing_counts1[9]] ./ [missing_counts2[2],missing_counts2[5],missing_counts2[9]],digits=2)

# ratio_cut = ratio_missing[1];
# ratio_depth = ratio_missing[2];
# ratio_y = ratio_missing[3];

# println("Le ratio de cut dans les 20% d'erreur les plus grandes: $ratio_cut.")
# println("Le ratio de depth dans les 20% d'erreur les plus grandes: $ratio_depth.")
# println("Le ratio de y dans les 20% d'erreur les plus grandes: $ratio_y.")

#pour le moment, parfaitement distribué, pcq on considère par les colonnes avec des valeurs manquantes, à vérifier une fois que les changements auront été effectués. 


In [768]:
# Afin de vérifier que les plus grosses erreurs ne sont pas provoquées par les valeurs manquantes
# On regarde dans le 1/5 des erreurs les plus grande. Si ratio_missing >> 0.2, il y a un problème

# num_zeros = count(x -> x == 0, skipmissing(valid2[observation_croiss[1:div(size(valid2,1),10)],:]))
# num_zeros2 = count(x -> x == 0, skipmissing(valid2))

# ratio_0 = round(num_zeros/num_zeros2,digits=2)
# # Print the result
# println("Le ratio de zéro dans les 10% d'erreur les plus grandes: $ratio_0.") #67% des zéros sont dans les dix pourcents les plus élevés, il faudra les remplacer


In [769]:
# Mat_train = Matrix(train₁)'
# k = 4
# Mat_test = Matrix(test₁)'

# kdtree = KDTree(Mat_train)
# idxs, dists = knn(kdtree, Mat_test, k, true)

# prediction = zeros(size(idxs))
# for i in 1:size(idxs,1)
#     prediction[i] =  mean(Matrix(price_train)[idxs[i]])
# end

# CSV.write("./results/predictions_knn.csv", DataFrame(ID = test_ids.ID, price = prediction));